## 02 Table - Carbon Inequality

In [2]:
import os
os.chdir('/Users/schoens/Documents/PhD/IIASA/04_Code/income_decile')

In [3]:
from pathlib import Path

import pandas as pd 
import numpy as np
import geopandas as gpd 
from shapely import box
import pycountry

from tqdm.notebook import tqdm

import config.config as cfg 
from config.config import (SCALING_DATA_DIR,
                           GRAPHICS_PAPER_APPENDIX_DIR, 
                           RESULTS_DIR,
                           regions,
                           region_naming_dict, 
                           population_shares, 
                           WID_DATA_PROCESSED
                           )

from tabulate import tabulate 

In [4]:
carbon_inequality_df = pd.DataFrame(columns = ['region', 'group',  'total_emission_share', 'equal_emission_share', 'percentage'])

region_to_global_df = pd.read_csv(SCALING_DATA_DIR / 'region_to_global_scaling_coefficients.csv', sep = ';', index_col = 0).mean(axis = 0)

within_region_df = pd.DataFrame(columns = ['p0p100', 'p0p50', 'p50p90', 'p90p100', 'p99p100', 'p999p100'], 
                                index = ['World'] + regions)

for region in ['World'] + regions:
    within_region_df.loc[region] = pd.read_csv(SCALING_DATA_DIR / 'within_countries' / f'{region}_scaling.csv', index_col = 0).iloc[:, 1:].mean(axis = 0).loc[within_region_df.columns]

In [5]:
results_df = pd.DataFrame(columns = ['region', 'group', 'total_emission_share', 'equal_emission_share', 'percentage'])
group_names = ['all', 'bottom 50', 'middle 40', 'top 10', 'top 1', 'top 0.1']
population_shares_all = [1] + population_shares
group_sizes = [1, 0.5, 0.4, 0.1, 0.01, 0.001]
counter = 0
for i_region, region in enumerate(['World'] + regions):
    for i_group, group in enumerate(['p0p100', 'p0p50', 'p50p90', 'p90p100', 'p99p100', 'p999p100']):
        total_emission_share = region_to_global_df.loc[region] * within_region_df.loc[region, group]
        equal_emission_share = region_to_global_df.loc['World'] * population_shares_all[i_region] * group_sizes[i_group]
        percentage = total_emission_share / equal_emission_share
        results_df.loc[counter,:] = [region, group_names[i_group], total_emission_share, equal_emission_share, percentage]
        counter += 1
results_df.index = results_df.region
results_df = results_df.drop(columns = 'region')

results_df.to_csv(RESULTS_DIR / 'carbon_inequality.csv')

In [45]:
Path(GRAPHICS_PAPER_APPENDIX_DIR / 'tables').mkdir(parents=True, exist_ok=True)

tmp_df = results_df.copy()
tmp_df.iloc[:, 1:] = (tmp_df.iloc[:, 1:].astype(float)*100).round(decimals = 2).copy()

with open(GRAPHICS_PAPER_APPENDIX_DIR / 'tables' / 'Carbon_Inequality.txt', 'w') as f:
    f.write(tabulate(tmp_df, tablefmt='latex'))